## groupby and merge in vaex on data larger than momory 
### good for single node large operation

In [2]:
# !pip install --upgrade vaex --quiet

In [1]:
import vaex

In [2]:
from google.cloud import storage
import os
bucket_name = os.environ['HYPERPLANE_GCP_BUCKET']
google_client = storage.Client()
blobs = google_client.list_blobs(bucket_name, prefix='data/crypto2')
file_size = [blob.size for blob in blobs]
print(f'number of files {len(file_size)}')
print(f'per file size {file_size[0]/1024/1024} MB')
print(f'total file compressed {file_size[0]/1024/1024/1024 * len(file_size)} GB')

number of files 4168
per file size 30.512420654296875 MB
total file compressed 124.19508719444275 GB


In [19]:
# df_one_file = pd.read_parquet("gs://shakdemo-hyperplane/data/crypto2/0.parquet") 
# print(f"one file uncompressed {df_one_file.memory_usage().sum()/1024./1024./1024.} G")
# print(f"total size uncompressed {df_one_file.memory_usage().sum()/1024./1024./1024. * len(file_size)} G")

## one file uncompressed 0.11138629913330078 G
## total size uncompressed 39.096590995788574 G

one file uncompressed 0.11138629913330078 G
total size uncompressed 39.096590995788574 G


In [8]:
num_files = int(20/0.11138629913330078)
num_files

179

In [9]:
%%time
df = vaex.open([f'gs://shakdemo-hyperplane/data/crypto2/{f}.parquet' for f in range(num_files)], progress=True)
df.shape

CPU times: user 13.7 s, sys: 979 ms, total: 14.7 s
Wall time: 1min 4s


(205850000, 13)

In [10]:
%%time
df.head(2)

CPU times: user 2.24 ms, sys: 26 µs, total: 2.26 ms
Wall time: 1.69 ms


#,info,timestamp,datetime,symbol,id,order,type,side,takerOrMaker,price,amount,cost,fee
0,"""{'M': True, 'T': '1617235200084', 'a': '6597508...",1617235200084,2021-04-01T00:00:00.084Z,BTC/USDT,659750817,0,0,buy,0,58739.5,0.006,352.437,0
1,"""{'M': True, 'T': '1617235200144', 'a': '6597508...",1617235200144,2021-04-01T00:00:00.144Z,BTC/USDT,659750818,0,0,buy,0,58740.6,0.002321,136.337,0


In [4]:
%%time
df.groupby(df[['side','symbol','timestamp']], 
           agg=[vaex.agg.sum('fee'), vaex.agg.mean('price')],
           progress = True)

groupby/binby [########################################] 100.00% elapsed time  :    82.99s =  1.4m =  0.0h    
 CPU times: user 7min 41s, sys: 39.2 s, total: 8min 20s
Wall time: 1min 24s


#,side,symbol,timestamp,fee_sum,price_mean
0,sell,BTC/USDT,1617235202409,0.0,58741.89999999993
1,sell,BTC/USDT,1617235209336,0.0,58755.970000000016
2,sell,BTC/USDT,1617235210124,0.0,58755.970000000016
3,sell,BTC/USDT,1617235210157,0.0,58755.970000000016
4,sell,BTC/USDT,1617235211251,0.0,58755.98000000035
...,...,...,...,...,...
"505,083",buy,BTC/USDT,1617274762525,0.0,58603.90000000018
"505,084",buy,BTC/USDT,1617274762692,0.0,58605.25999999983
"505,085",buy,BTC/USDT,1617274786884,0.0,58613.11999999993
"505,086",buy,BTC/USDT,1617274790385,0.0,58617.05000000003


In [6]:
%%time 
avg_vols_by_time = df.groupby(df['timestamp'], agg = [vaex.agg.mean('price')], progress = True)
avg_vols_by_time

groupby/binby [########################################] 100.00% elapsed time  :    20.61s =  0.3m =  0.0h
 CPU times: user 2min 6s, sys: 14.3 s, total: 2min 20s
Wall time: 20.6 s


#,timestamp,price_mean
0,1617235202409,58741.89999999993
1,1617235205646,58752.44000000011
2,1617235209336,58755.970000000016
3,1617235210124,58755.970000000016
4,1617235210157,58755.970000000016
...,...,...
"503,317",1617274767362,58611.99000000017
"503,318",1617274786884,58613.11999999993
"503,319",1617274790385,58617.05000000003
"503,320",1617274793670,58617.61999999993


In [11]:
%%time
df.join(avg_vols_by_time, left_on = 'timestamp', right_on = 'timestamp', lsuffix = 'l', rsuffix = 'r')

CPU times: user 1min 9s, sys: 6.53 s, total: 1min 16s
Wall time: 12.3 s


#,info,timestampl,datetime,symbol,id,order,type,side,takerOrMaker,price,amount,cost,fee,timestampr,price_mean
0,"""{'M': True, 'T': '1617235200084', 'a': '6597508...",1617235200084,2021-04-01T00:00:00.084Z,BTC/USDT,659750817,0.0,0.0,buy,0.0,58739.46,0.006,352.43676,0.0,1617235200084,58739.46000000051
1,"""{'M': True, 'T': '1617235200144', 'a': '6597508...",1617235200144,2021-04-01T00:00:00.144Z,BTC/USDT,659750818,0.0,0.0,buy,0.0,58740.55,0.002321,136.33681655,0.0,1617235200144,58740.55000000038
2,"""{'M': True, 'T': '1617235200345', 'a': '6597508...",1617235200345,2021-04-01T00:00:00.345Z,BTC/USDT,659750819,0.0,0.0,buy,0.0,58740.55,0.002537,149.02477535,0.0,1617235200345,58740.55000000038
3,"""{'M': True, 'T': '1617235200355', 'a': '6597508...",1617235200355,2021-04-01T00:00:00.355Z,BTC/USDT,659750820,0.0,0.0,sell,0.0,58740.54,0.001674,98.33166396,0.0,1617235200355,58740.53999999949
4,"""{'M': True, 'T': '1617235200405', 'a': '6597508...",1617235200405,2021-04-01T00:00:00.405Z,BTC/USDT,659750821,0.0,0.0,buy,0.0,58740.55,0.01674,983.316807,0.0,1617235200405,58740.55000000038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"403,649,995","""{'M': True, 'T': '1617235202708', 'a': '6597508...",1617273800943,2021-04-01T10:43:20.943Z,BTC/USDT,660317062,0.0,0.0,buy,0.0,58802.48,0.006587,387.33193576,0.0,1617273800943,58802.48000000035
"403,649,996","""{'M': True, 'T': '1617235202720', 'a': '6597508...",1617273801048,2021-04-01T10:43:21.048Z,BTC/USDT,660317063,0.0,0.0,buy,0.0,58802.48,0.757028,44515.12382944,0.0,1617273801048,58802.48000000035
"403,649,997","""{'M': True, 'T': '1617235202753', 'a': '6597508...",1617273801103,2021-04-01T10:43:21.103Z,BTC/USDT,660317064,0.0,0.0,buy,0.0,58802.48,0.0048,282.251904,0.0,1617273801103,58802.48000000035
"403,649,998","""{'M': True, 'T': '1617235202760', 'a': '6597508...",1617273801149,2021-04-01T10:43:21.149Z,BTC/USDT,660317065,0.0,0.0,sell,0.0,58803.03,0.000616,36.22266648,0.0,1617273801149,58803.029999999984
